In [ ]:
## Import necessary libraries here (You can add libraries you want to use here)
import cv2
import numpy as np
from scipy.io import loadmat
from scipy import ndimage
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import copy
import os
import time
%matplotlib inline

# Part 1: A Feature Tracker (50 Points)

## Overview

In the problem, you will implement a corner detector and feature tracker that track features from the image sequence hotel. Since this is a two part problem, we have included precomputed intermediate results in the *Data* section in case you’re unable to complete any portion.

<img src="https://drive.google.com/uc?id=1sBtKpU2mYEPZ9c2Cvw-DBuLBPK2gYwC-" width="700"/>

**Note:**  Do not use existing keypoint
detectors, trackers, or structure from motion code, such as found on the web, and OpenCV.

## Data

**WARNING: Colab deletes all files everytime runtime is disconnected. Make sure to re-download the inputs when it happens.**

In [ ]:
# Download Data -- run this cell only one time per runtime
if not os.path.exists('/content/part1_images/hotel.seq41.png'):
  !gdown 1fT0H-FbbDZnjMfCJHZcscpcwAXHhGgNw
  !unzip "/content/part1_images.zip" -d "/content/"
  !gdown 1r-Pdino6MRLCEWX_sQOgd8D5AVsRc7Ym
  # Load Initial Key Points
  data = loadmat('/content/initial_keypoints.mat')
  X0 = data['Xo']
  Y0 = data['Yo']

## Helper Functions

In [89]:
def readImages(folder, num_images):
  arr_images = []
  for i in range(num_images):
    arr_images.append(cv2.imread(f'{folder}hotel.seq{i}.png'))
  return np.array(arr_images, dtype=np.float32)

def compute_H(patch): #Given a patch, compute the 2nd moment matrix within the patch
  Ix = patch[:, 1:] - patch[:, :-1]
  Iy = patch[1:, :] - patch[:-1, :]
  Ix = Ix[:-1, :]
  Iy = Iy[:, :-1]
  Ix_sq = np.sum(np.square(Ix))
  Iy_sq = np.sum(np.square(Iy))
  Ix_Iy = np.sum(Ix * Iy)
  H = np.array([[Ix_sq, Ix_Iy], [Ix_Iy, Iy_sq]])
  return H, Ix, Iy

def pad_the_patch(patch): #pad the image by replicating the last row & last column. Resultant patch will have 1 additional row & column than the original one.
  patch = np.concatenate((patch, np.expand_dims(patch[:, -1], axis=1)), axis=1)
  patch = np.concatenate((patch, np.expand_dims(patch[-1, :], axis=0)), axis=0)
  return patch

# read all 51 sequences of images
folder = '/content/part1_images/'
im = readImages(folder, 51)


## 1.1 Keypoint Selection (15 pts)

For the first frame, use the second moment matrix to locate strong corners to use as keypoints.
These points will be tracked throughout the sequence in the second part of the problem. Choose a proper threshold so that edges and noisy patches are ignored. Do local non-maxima suppression over a 5x5 window centered at each point.
This should give several hundred good points to track.

### Code (5 pts)

In [ ]:
def apply_NMS(f_values, window_size):
  '''
  Apply local Non-Maxima Suppression:
  1) Slide a 5x5 window all over the image.
  2) Retain the center of those 5x5 windows as keypoints in which the center pixel is the local maxima in that window.
  '''
  (rows, cols) = f_values.shape
  half_size = window_size//2
  keypoints = np.zeros((rows, cols))
  for i in range(half_size, rows-half_size):
    for j in range(half_size, cols-half_size):
      patch = f_values[i-half_size:i+half_size+1, j-half_size:j+half_size+1]
      uniq, counts = np.unique(patch, return_counts=True)
      if counts[-1] > 1:
        continue
      if uniq[-1] == f_values[i, j]:
        keypoints[i, j] = 255
  return keypoints

def getKeypoints(img, tau):
  '''
  Detecting keypoints using Harris corner criterion
  img: input image
  tau: threshold 
  
  output: (N,2) array of [x,y] keypoints (x & y are 0-indexed with upper left pixel of the image being (0,0). i.e, OpenCV frame convention).
  '''
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  (rows, cols) = img.shape
  window_size = 5 
  half_size = window_size//2
  f_values = np.zeros_like(img)
  for i in range(half_size, rows-half_size):
    for j in range(half_size, cols-half_size):
      patch = img[i-half_size:i+half_size+1, j-half_size:j+half_size+1]
      padded_patch = pad_the_patch(patch)
      H, _, _ = compute_H(padded_patch)
      harris_operator = np.linalg.det(H) / np.trace(H)
      if harris_operator>tau:
        f_values[i, j] = harris_operator
  f_values = f_values/np.amax(f_values)
  keypoints = apply_NMS(f_values, window_size)
  print('Number of keypoints selected:', np.sum(keypoints)/255)

  keypoints_opencv = []
  for i in range(half_size, rows-half_size):
    for j in range(half_size, cols-half_size):
      if keypoints[i,j] == 255:
        keypoints_opencv.append([j, i])
  return keypoints_opencv

tau = 3.15*255
# tau is tuned as below: <tau> -> <number of keypoints it gave>
# 0.17->1102 0.25->1061 0.55->991 0.85->865 0.95->813 1.15->740 3.15->201
keypoints_opencv = getKeypoints(im[0], tau)

# add plots for the write-up
display_img = cv2.cvtColor(im[0], cv2.COLOR_BGR2GRAY)
(rows, cols) = display_img.shape
display_img = cv2.cvtColor(display_img, cv2.COLOR_GRAY2BGR)
for key_pt in keypoints_opencv:
  display_img = cv2.circle(display_img, (key_pt[0], key_pt[1]), radius=3, color=(0, 255, 0), thickness=-1)
cv2_imshow(display_img)

### Write-up (10 pts)


*   (5 pts) Explain your implementation of getKeypoints()
*   (5 pts) Display the first frame of the sequence overlaid with the detected keypoints. Ensure that they are clearly visible (plot with `color='g' and linewidths=3`)



#### Description of getKeypoints():
- A 5x5 window is slid on the grayscale image (with a stride of 1) & the 2nd moment matrix `H` is calculated using the below formula:

  - <img src="https://drive.google.com/uc?id=1_W_a9FwMjDlyTWAXLOrXvsFBcIEeZfli" height=400 align="center"/>
  - where:
    - `i_x` & `i_y` are the gradients in x & y directions.
    - The gradient `i_x` is calculated as `intensity of pixel (p+1,q) - intensity of pixel (p,q)`
    - The summations in the formulas for `A`,`B`&`C` run over the 5x5 window.

- The harris operator is calculated by the below formula:
  - <img src="https://drive.google.com/uc?id=1gBxEZQ1yttjpwMHdpNpbFhq4kZlQJPUr" height=200 align="center"/>
  - It is calculated over all the 5x5 windows.
- ##### Thresholding:
  - Those pixels for which harris operator yielded a value of greater than `tau` were marked as potential keypoints & the rest are discarded.
  - `tau = 803.25` was selected after a bit of tuning.
- ##### Normalization:
  - Then, the resultant harris operator matrix (of the same size as image) is divided by its maximum value.
- Finally, non-maxima suppression is done (in the below way):
  - If a pixel is strictly greater than every other pixel in the 5x5 neighborhood centered around it, it is retained. Else, it is discarded.
- Note:
  - No padding is done on the original image.
  - Due to this, no keypoints will be detected in the `window_size//2` = 2 columns or rows along the border.

#### Keypoints:
- The below image shows the final keypoints obtained by following the above method.
- `201` keypoints were selected for `tau = 803.25`.
- <img src="https://drive.google.com/uc?id=1thCSU_TWlgDcGmzrcDMjZ43CszK2cQl3" align="center"/>

## 1.2 Feature Tracking (35 pts)

Apply the Kanade-Lucas-Tomasi tracking procedure to track the keypoints found in part 1.1 (or the given keypoints in the *Data* section) throughout the hotel sequence. 

<img src="https://drive.google.com/uc?id=1dU4p4YcnXoQFnrNvleEty_4tDECkVW9Q" width="500"/>

Some keypoints will move out of the image frame over the course of the sequence. Discard any track if the predicted translation falls outside the image frame.

**Hint:**

*  From the 1st frame to the 2nd frame, use the detected keypoints at the first frame as initialization points. From the 2nd to 3rd frame, use the tracked positions at the 2nd frame as initialization. Note that the tracked keypoints in general are not at integer positions.

*  For each point, use a window size of 15 x 15.

Add codes to **plot** your tracked points overlayed in the **first sequence** and the **last sequence**. They should look similar to the second picture shown in the Overview section. 



### Code (10 pts)

In [ ]:
def trackPoints(pt_x, pt_y, im, ws, out_of_bound_pts):
  '''
  Tracking initial points (pt_x, pt_y) across the image sequence
  Outputs:
    track_x: [Number of keypoints] x [2]
    track_y: [Number of keypoints] x [2]
  '''
  pt_x = pt_x.astype('float32')
  pt_y = pt_y.astype('float32')
  N = np.prod(pt_x.shape)
  nim = len(im)
  track_x = np.zeros((N, nim))
  track_y = np.zeros((N, nim))
  track_x[:,0] = pt_x
  track_y[:,0] = pt_y
  if len(im.shape) == 4:
    im = im[:, :, :, 0].astype('float32')
  else:
    input('No resizing is being done. Continue?')
  for t in range(nim-1):
    track_x[:, t+1], track_y[:, t+1] = getNextPoints(track_x[:, t], track_y[:, t], im[t,:,:], im[t+1,:,:], ws, out_of_bound_pts)

  return np.concatenate((np.expand_dims(track_x, axis=2), np.expand_dims(track_y, axis=2)), axis=2)

def getNextPoints(xs, ys, im1, im2, ws, out_of_bound_pts):
  '''
  Iterative Lucas-Kanade feature tracking
  x,  y : initialized keypoint position in im2
  ws: patch window size

  output: tracked keypoint positions in im2
  '''
  threshold = 0.5
  x_next_frame = []
  y_next_frame = []
  (rows, cols) = im1.shape
  for idx, (x,y) in enumerate(zip(xs, ys)):
    u = 10
    v = 10
    x_dash = copy.copy(x)
    y_dash = copy.copy(y)
    patch = cv2.getRectSubPix(im1, (ws,ws), (x,y))
    H, Ix, Iy = compute_H(pad_the_patch(patch))
    while (u>threshold) or (v>threshold):
      It = cv2.getRectSubPix(im2, (ws,ws), (x_dash,y_dash)) - cv2.getRectSubPix(im1, (ws,ws), (x,y))
      Ix_It = -1 * (np.sum(Ix*It))
      Iy_It = -1 * (np.sum(Iy*It))
      b = [Ix_It, Iy_It] # To solve ax=b, a=H & b is this matrix
      try:
        [u, v] = np.linalg.solve(H, b)
      except:
        print(np.linalg.det(H))
        input('stopping')
      new_x_dash = x_dash + u
      new_y_dash = y_dash + v
      if new_y_dash<0 or new_y_dash>rows-1 or new_x_dash<0 or new_x_dash>cols-1:
        out_of_bound_pts.append(idx)
        break
      else:
        x_dash = new_x_dash
        y_dash = new_y_dash
    x_next_frame.append(x_dash)
    y_next_frame.append(y_dash)
  return x_next_frame, y_next_frame

ws = 7
keypoints_list = copy.copy(np.array(keypoints_opencv))
out_of_bound_pts = []
tracked_pts = trackPoints(pt_x=keypoints_list[:,0], pt_y=keypoints_list[:,1], im=im, ws=ws, out_of_bound_pts=out_of_bound_pts)

# plot your results
display_img1 = copy.copy(im[0])
display_img2 = copy.copy(im[0])
display_img3 = copy.copy(im[0])
rand_20_tracked_pts = np.random.permutation(np.arange(tracked_pts.shape[0]))[:20]
for i in range(tracked_pts.shape[0]):
  tracked_pt = tracked_pts[i]
  display_img1 = cv2.circle(display_img1, (int(tracked_pt[0][0]), int(tracked_pt[0][1])), radius=2, color=(0, 255, 0), thickness=-1)
  display_img1 = cv2.circle(display_img1, (int(tracked_pt[1][0]), int(tracked_pt[1][1])), radius=2, color=(0, 0, 255), thickness=-1)
  if i in rand_20_tracked_pts:
    for consec_frame_pts in tracked_pt:
      display_img2 = cv2.circle(display_img2, (int(consec_frame_pts[0]), int(consec_frame_pts[1])), radius=2, color=(0, 0, 255), thickness=-1)
  if i in out_of_bound_pts:
    for consec_frame_pts in tracked_pt:
      display_img3 = cv2.circle(display_img3, (int(consec_frame_pts[0]), int(consec_frame_pts[1])), radius=2, color=(0, 0, 255), thickness=-1)

cv2_imshow(display_img1)
cv2_imshow(display_img2)
cv2_imshow(display_img3)

### Write-up (25 pts)

*   (5 pts) For all the keypoints, display (1) the keypoints at the first frame (as green) and (2) the tracked keypoints at the second frame (as red) on the first frame of the sequence.
*   (10 pts) For 20 random keypoints, draw the 2D path over the sequence of frames. That is, plot the progression of image coordinates for each of the 20 keypoints. Plot each of the paths on the same figure, overlaid on the first frame of the sequence.
*   (10 pts) On top of the first frame, plot the points which have moved out of frame at some point along the sequence.





- <img src="https://drive.google.com/uc?id=1kz8e8M1_RTHjEpZVAtdn8rew49lDUsbI" height=400 align="center"/>

- <img src="https://drive.google.com/uc?id=1SbVyEFGKRlysslutuPNzkoj2rB0Y12c1" height=400 align="center"/>

- <img src="https://drive.google.com/uc?id=1aRwEDLJ39ZgMVjPk8F1xHjHQ5UZEzRMG" height=400 align="center"/>


# Part 2: Shape Alignment (30 Points)

## Overview
In this problem, you will write a function that aligns two sets of points using global image transformation (similarity, affine, or perspective) and returns $T$  where $T$ is a transformation that maps non-zero points in $im1$ to non-zero points in $im2$. You may choose the alignment algorithm and the type of (global) transformation (e.g., rigid Euclidean, affine, perspective).


<img src="https://drive.google.com/uc?id=1PnWIy9ZdP9SGkmGNtFCJ-JzKLwmW-qaN" width="1000"/>

Test your code on the 25 image pairs provided in the supplementary material. We have included functions 
**(will check) evalAlignmentAll and displayAlignment to help with evaluation and display**.




## Data

**WARNING: Colab deletes all files everytime runtime is disconnected. Make sure to re-download the inputs when it happens.**

In [ ]:
# Download Data -- run this cell only one time per runtime
!gdown 18Px9uQyY1fGGyEAQhzt3h4yDQonU_Sgm
!unzip "/content/part2_images.zip" -d "/content/"

Downloading...
From: https://drive.google.com/uc?id=18Px9uQyY1fGGyEAQhzt3h4yDQonU_Sgm
To: /content/part2_images.zip
100% 78.4k/78.4k [00:00<00:00, 69.5MB/s]
Archive:  /content/part2_images.zip
   creating: /content/part2_images/
 extracting: /content/part2_images/Bone_1.png  
 extracting: /content/part2_images/elephant_1.png  
 extracting: /content/part2_images/brick_2.png  
 extracting: /content/part2_images/Heart_2.png  
 extracting: /content/part2_images/Bone_2.png  
  inflating: /content/part2_images/elephant_2.png  
 extracting: /content/part2_images/brick_1.png  
 extracting: /content/part2_images/Heart_1.png  
  inflating: /content/part2_images/device7_1.png  
  inflating: /content/part2_images/device7_2.png  
 extracting: /content/part2_images/fork_2.png  
 extracting: /content/part2_images/turtle_2.png  
  inflating: /content/part2_images/fork_1.png  
 extracting: /content/part2_images/turtle_1.png  
 extracting: /content/part2_images/butterfly_2.png  
 extracting: /content/pa

## Helper Functions

## Code (15 pts)

In [ ]:
def initialize(im1, mean1, mean2, scale_fact_1_to_2, im2, rows, cols):
  '''
  Apply translation & scaling to im1 as a way of initialization
  '''
  # Translation
  old_img = copy.copy(im1)
  im1 = (mean2-mean1) + im1

  # Scaling
  new_mean = np.mean(im1, axis=0)
  im1 = im1 - new_mean
  im1 = im1 * scale_fact_1_to_2
  im1 = im1 + new_mean
  im1[:, 0] = np.clip(im1[:, 0], 0, rows-1)
  im1[:, 1] = np.clip(im1[:, 1], 0, cols-1)
  return im1

def find_NN(i, im2):
  distance = 1e8
  for j in im2:
    curr_distance = np.linalg.norm(i-j)
    if curr_distance < distance:
      distance = curr_distance
      nn = j
  return nn

def warp_image(im1, m1, m2, m3, m4, t1, t2, rows, cols):
  warped_im1 = []
  for i in im1:
    warped_im1.append([np.clip(m1*i[0]+m2*i[1]+t1, 0, rows-1), np.clip(m3*i[0]+m4*i[1]+t2, 0, cols-1)])
  return warped_im1

def align_shape(im1, im2, num_iter=50, show_every_iter=False):
  '''
  im1: input edge image 1
  im2: input edge image 2

  Output: transformation T [3] x [3]
  '''
  orig_im2 = copy.copy(im2)
  (rows, cols) = im1.shape
  idx_grid = np.mgrid[0:rows, 0:cols]
  idx_grid = np.swapaxes(idx_grid, 0, 1)
  idx_grid = np.swapaxes(idx_grid, 1, 2)
  im1 = idx_grid[im1==255] # Gives coordinates of pixels in (row, col) format
  im2 = idx_grid[im2==255]
  mean1 = np.mean(im1, axis=0)
  mean2 = np.mean(im2, axis=0)
  scale1 = np.std(im1, axis=0)
  scale2 = np.std(im2, axis=0)
  scale_fact_1_to_2 = scale2 / scale1
  im1 = initialize(im1, mean1, mean2, scale_fact_1_to_2, im2, rows, cols)

  for iter in range(num_iter):
    matches = []
    for i in im1:
      nn_of_i = find_NN(i, im2)
      matches.append([i, nn_of_i])
    A = []
    B = []
    for match in matches:
      x = match[0][0]
      y = match[0][1]
      x_dash = match[1][0]
      y_dash = match[1][1]
      A.append([x, y, 0, 0, 1, 0])
      A.append([0, 0, x, y, 0, 1])
      B.append(x_dash)
      B.append(y_dash)
    m1, m2, m3, m4, t1, t2 = np.linalg.lstsq(A, B, rcond=None)[0]
    im1 = warp_image(im1, m1, m2, m3, m4, t1, t2, rows, cols)
    if (show_every_iter):
      test = copy.copy(orig_im2)
      for i in range(len(im1)):
        test[int(im1[i][0]), int(im1[i][1])] = 255
      cv2_imshow(test)
  return im1

In [ ]:
def evalAlignment(aligned1, im2):
  '''
  Computes the error of the aligned image (aligned1) and im2, as the
  average of the average minimum distance of a point in aligned1 to a point in im2
  and the average minimum distance of a point in im2 to aligned1.
  '''
  d2 = ndimage.distance_transform_edt(1-im2) #distance transform
  err1 = np.mean(np.mean(d2[aligned1 > 0]))
  d1 = ndimage.distance_transform_edt(1-aligned1);
  err2 = np.mean(np.mean(d2[im2 > 0]))
  err = (err1+err2)/2;
  return err

def displayAlignment(im1, im2, aligned1, thick=False):
  '''
  Displays the alignment of im1 to im2
     im1: first input image to alignment algorithm (im1(y, x)=1 if (y, x) 
      is an original point in the first image)
     im2: second input image to alignment algorithm
     aligned1: new1(y, x) = 1 iff (y, x) is a rounded transformed point from the first time 
     thick: true if a line should be thickened for display
  ''' 
  if thick:
    # for thick lines (looks better for final display)
    dispim = np.concatenate((cv2.dilate(im1.astype('uint8'), np.ones((3,3), np.uint8), iterations=1), \
                             cv2.dilate(aligned1.astype('uint8'), np.ones((3,3), np.uint8), iterations=1), \
                             cv2.dilate(im2.astype('uint8'), np.ones((3,3), np.uint8), iterations=1)), axis=-1)
  else:
    # for thin lines (faster)
    dispim = np.concatenate((im1, aligned1, im2), axis = -1)
  return dispim
  

In [ ]:
imgPath = '/content/part2_images/';

objList = ['apple', 'bat', 'bell', 'bird', 'Bone', 'bottle', 'brick', \
    'butterfly', 'camel', 'car', 'carriage', 'cattle', 'cellular_phone', \
    'chicken', 'children', 'device7', 'dog', 'elephant', 'face', 'fork', 'hammer', \
    'Heart', 'horse', 'jar', 'turtle']

numObj = len(objList)

for idx in range(11, numObj):
  start_time = time.time()
  im1 = cv2.imread(imgPath+objList[idx]+'_1.png', 0)
  im2 = cv2.imread(imgPath+objList[idx]+'_2.png', 0)
  (rows, cols) = im1.shape

  aligned_im1_coords = align_shape(im1, im2)
  end_time = time.time()
  aligned_im1_coords = np.array(aligned_im1_coords)
  aligned_im1 = np.zeros((rows, cols))
  for i in range(np.shape(aligned_im1_coords)[0]):
    aligned_im1[int(aligned_im1_coords[i][0]), int(aligned_im1_coords[i][1])] = 255
  print('Runtime for ', objList[idx], '= ', end_time - start_time, ' seconds')
  error = evalAlignment(aligned_im1, im2)
  dispim = displayAlignment(im1, im2, aligned_im1, thick=True)
  print('Error for ', objList[idx], '= ', error)
  cv2_imshow(dispim)
  


## Write-up (15 pt)

1. (5 pts) Give a brief explanation of your algorithm, initialization, and model of the transformation.

2. (10 pts) For each result, give:
  1.   The alignment display
  2.   The final error
  3.   The runtime






#### 1) Algorithm, initialization & Transformation model:

##### - Algorithm:
- Firstly, the edge points in both the images are calculated (by storing the coordinates of the pixels whose value is 255).
- Secondly, these arrays of edge points are used to generate an initial transformation as described in the next sub-section.
- Then, the Iterative Closest Point algorithm is applied in the following way:
  - Determine the match for a point in image1 by finding the point in image2 which has the smallest Euclidean distance. Repeat this for all the points in image1.
  - Get the transformation that satisfies these pairs of matches as close as possible as described in the 'Transformation model' sub-section below.
  - Using the estimated transformation matrix, warp all the points in image1.
  - Repeat the above steps feeding the output of one iteration as the input for the next iteration.
- The output of the final iteration gives the final version of image1 aligned with image2.

##### - Initialization:
  - The first image is translated & scaled in the following way to make it look closer to the second image.
  - First, the difference in the centroids between both the images is calculated & all the points in image1 are translated by this value. Let's call this image `translated_img`.
  - Then, the standard deviation of each image is calculated & the scale factor is defined as `std_dev of image2 / std_dev of image1`.
  - Finally, scaling is performed by radially stretching (from centroid) every point in `translated_img` by this `scale factor`.
  - <ins>Note</ins>: 'image' in this sub-section referes to the array of edge points in a picture.

##### - Transformation model:
  - An affine transformation model is used to fit the pairs of matches between both the images.
  - This is done using Direct Linear transformation.
  - A Least squares approach is used to find the parametres of the transformation.
  - <ins>Note</ins>: 'image' in this sub-section refers to the array of edge points in a picture.

#### 2) Results:
- <img src="https://drive.google.com/uc?id=19IrJ3AsmIOTIHt-fVArm0QxqNOF6hlhc" align="center"/>

  - Runtime:194
  - Error:174
- <img src="https://drive.google.com/uc?id=106IkD29-eYkdbj-hKnsCFebqGlgKjwu-" align="center"/>

  - Runtime:1183
  - Error:425
- <img src="https://drive.google.com/uc?id=1M1tDu-_0yUv4-3E_GRUd8B44m4Nk8X8x" align="center"/>

  - Runtime:125
  - Error:179
- <img src="https://drive.google.com/uc?id=1HAGqeu1c-7OtRKgBff5ALAGFNLPmnwvi" align="center"/>

  - Runtime:469
  - Error:221
- <img src="https://drive.google.com/uc?id=1z9K5JhKtOJtCbEugL9OsHSfwaKfJrLCz" align="center"/>

  - Runtime:528
  - Error:320
- <img src="https://drive.google.com/uc?id=1Q-hvSrUirBOOO-2rlIG8eyIs2rCo8b3c" align="center"/>

  - Runtime:49
  - Error:246
- <img src="https://drive.google.com/uc?id=1BzhlGskNeLPRr-fS6u9PtwheGVIKjMDd" align="center"/>

  - Runtime:115
  - Error:303
- <img src="https://drive.google.com/uc?id=1Bu-wYZDRw_-Vg4fdV3wT1kKVZ_lGaBXc" align="center"/>

  - Runtime:464
  - Error:212
- <img src="https://drive.google.com/uc?id=1aDSvtY_TfkCEgBJ1KqCMYyCR-adaUX8Y" align="center"/>

  - Runtime:868
  - Error:269
- <img src="https://drive.google.com/uc?id=1KPNNB3MMbnp9dn4ETaHhXgPsb7ewfcAp" align="center"/>

  - Runtime:91
  - Error:242
- <img src="https://drive.google.com/uc?id=1U0eas9COqcj2R7Zragppsb-BpSLdOrdz" align="center"/>

  - Runtime:211
  - Error:284
- <img src="https://drive.google.com/uc?id=1YzWMeo0QmWVkgCWzwWMU-L7pv06cFEm2" align="center"/>

  - Runtime:2742
  - Error:392
- <img src="https://drive.google.com/uc?id=1c8gtSRW2c6HF3D7-oiw5MZknJ18-kKDv" align="center"/>

  - Runtime:360
  - Error:258
- <img src="https://drive.google.com/uc?id=1FbAgczb6bzJ-nuzvC1xIRWBSNrYnjHZO" align="center"/>

  - Runtime:237
  - Error:144
- <img src="https://drive.google.com/uc?id=1T4w99rLLFI-e37_lseIOga-9zWuuzEiX" align="center"/>

  - Runtime:63
  - Error:155
- <img src="https://drive.google.com/uc?id=1aUcR-J3ViBLqDU2QBjBI7_rwqex4FHzh" align="center"/>

  - Runtime:5258
  - Error:382
- <img src="https://drive.google.com/uc?id=1Jyw2tEcqXja4YiWf6-7qV0vWRS9xp2-f" align="center"/>

  - Runtime:2141
  - Error:351
- <img src="https://drive.google.com/uc?id=1V3GtwterQUnx1wcCqVizxdCObSQgsPrQ" align="center"/>

  - Runtime:4196
  - Error:562
- <img src="https://drive.google.com/uc?id=1gpmsr3IOw91QtGz4u-ueMLH_YXwXUutm" align="center"/>

  - Runtime:222
  - Error:232
- <img src="https://drive.google.com/uc?id=1wHkWNe9WuflBibGXfP-PrgZyCS9NhUY_" align="center"/>

  - Runtime:1409
  - Error:357
- <img src="https://drive.google.com/uc?id=1V4lUww35Z6kerzIcy2JsJJLb0_JZ8Cbp" align="center"/>

  - Runtime:151
  - Error:156
- <img src="https://drive.google.com/uc?id=1l6YYA8EeLd2pPNLFM_Gfgi6qs5rW0rQp" align="center"/>

  - Runtime:677
  - Error:311
- <img src="https://drive.google.com/uc?id=1bYlV3GaT-yhm_kB9gMM0vPKaS30mHlvb" align="center"/>

  - Runtime:4387
  - Error:492
- <img src="https://drive.google.com/uc?id=1pXpbBho3Oq9pKGPypzebZVXechqKUNTa" align="center"/>

  - Runtime:1463
  - Error:386
- <img src="https://drive.google.com/uc?id=1hSmUT9s_iOAknjDx7peMoZK4vl4iqwQE" align="center"/>

  - Runtime:400
  - Error:250

<ins>Note</ins>: The above runtimes are in seconds.

# Part 3: Object Instance Recognition (20 points)

## Overview
This problem explores the Lowe-style object instance recognition.

Implement the nearest neighbor distance ratio test using the pre-computed SIFT features SIFT_features.mat provided in the supplementary material. The Frame1, Frame2 indicate the 2D position, scales, and the orientation of the descriptors and Descriptor1, Descriptor2 are the correspondin 128-D SIFT features. Display the matches like this:

<img src="https://drive.google.com/uc?id=1mULTvHYeP5uj_vi7nwWThBHkDbv1eSue" width="1000"/>



## Data

In [ ]:
# Download Data -- run this cell only one time per runtime
!gdown 10ByzpFbB-z178VGjwmCwc95wInD8vpNM # SIFT Features
!gdown 1KLWGMtDEMNNrmzd3Qezrs2-NQR52OfoU # Stop sign image 1
!gdown 13y-o1vdGN6CqqPuUcgU7pIxODTxrYS7J # Stop sign image 1

## Code (10 pts)

In [ ]:
def compare(x):
  return x[0]

img1 = cv2.imread('/content/stop1.jpg')
img2 = cv2.imread('/content/stop2.jpg')

## inside the sift are:
## Descriptor1, Descriptor2: SIFT features from image 1 and image 2
## Frame1, Frame2: position, scale, rotation of keypoints
data = loadmat('/content/SIFT_features.mat')
Frame1 = data['Frame1']
Descriptor1 = data['Descriptor1']
Frame2 = data['Frame2']
Descriptor2 = data['Descriptor2']

Frame1 = np.transpose(Frame1)
Frame2 = np.transpose(Frame2)
Descriptor1 = np.transpose(Descriptor1).astype('float32')
Descriptor2 = np.transpose(Descriptor2).astype('float32')


matches_NN = []
matches_2NN = []
threshold_NN = 120
threshold_2NN = 0.548
data_type = [['distance', 'float32'], ['idx', int]]
for i in range(Descriptor1.shape[0]):
  diff = Descriptor2 - Descriptor1[i]
  dist = np.expand_dims(np.linalg.norm(diff, axis=1), axis=1)
  temp = np.expand_dims([t for t in range(dist.shape[0])], axis=1)
  dist = np.concatenate((dist, temp), axis=1).tolist()
  dist.sort(key=compare)

  if dist[0][0] < threshold_NN:
    matches_NN.append([i, int(dist[0][1])])
  if (dist[0][0]/dist[1][0]) < threshold_2NN:
    matches_2NN.append([i, int(dist[0][1])])

## Display the matched keypoints
(rows1, cols1, _) = img1.shape
(rows2, cols2, _) = img2.shape
combined_img = np.zeros((max(rows1, rows2), cols1+cols2, 3))
combined_img[:rows1, :cols1, :] = img1
combined_img[:, cols1:cols1+cols2, :] = img2
combined_img2 = copy.copy(combined_img)

print(len(matches_NN), len(matches_2NN))
for match_NN in matches_NN:
  combined_img = cv2.line(combined_img, (int(Frame1[match_NN[0]][0]), int(Frame1[match_NN[0]][1])), (int(Frame2[match_NN[1]][0])+cols1, int(Frame2[match_NN[1]][1])), (0, 255, 0), 2)

for match_NN in matches_2NN:
  combined_img2 = cv2.line(combined_img2, (int(Frame1[match_NN[0]][0]), int(Frame1[match_NN[0]][1])), (int(Frame2[match_NN[1]][0])+cols1, int(Frame2[match_NN[1]][1])), (0, 255, 0), 2)

cv2_imshow(combined_img)
cv2_imshow(combined_img2)

## Write-up (10 pts)

(5 pts) Display:

1. the matches by thresholding nearest neighbor distances.

2. the matches by thresholding the distance ratio. 

(5 pts) Describe the differences of (1) and (2).

#### Display:
 - Matches by thresholding nearest neighbor distances:
 - <img src="https://drive.google.com/uc?id=1LxXW19UoMUcZb0zmVWYnU1rJh5tREOGh" align="center"/>
 
 - Matches by thresholding distance ratio:
 - <img src="https://drive.google.com/uc?id=1EJzAiMPLDfL5VSTBSlUtRp0bEGwdumvo" align="center"/>

#### Note:
  - The distance ratio method is better at telling apart close-looking but dissimilar features.
  - It has a better False-Positive rate.
  - It can be observed (as shown below) that the nearest neighbor distance method falsely matched 2 SIFT descriptors on the white border of the stop sign. But the distance ratio method didn't do such a mistake.
  - <img src="https://drive.google.com/uc?id=10vRqvAc6b3J651YPGjcsmMzLKciIoufH" align="center"/>
  